In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_vcm_dwh.d_mdq_pog 
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
AS
SELECT
  * EXCEPT(proc_date),
  'IN REVIEW' applied_external_report
FROM {catalog_name}.udp_wcm_silver_dataportal.d_mdq_pog
WHERE upload_time = (SELECT MAX(upload_time) FROM {catalog_name}.udp_wcm_silver_dataportal.d_mdq_pog);
""")

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog_name}.udp_wcm_gold_vcm_dwh.d_mdq_pog_his (
  calday DATE,
  store_concept STRING,
  product_id STRING,
  core_range STRING,
  mdq DOUBLE,
  approval_status STRING,
  upload_user STRING,
  upload_time TIMESTAMP,
  applied_external_report STRING
)
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
);
""")

In [0]:
spark.sql(f"""
DELETE FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_mdq_pog_his
WHERE calday = DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS);
""")

In [0]:
spark.sql(f"""
INSERT INTO {catalog_name}.udp_wcm_gold_vcm_dwh.d_mdq_pog_his
SELECT
    DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) calday,
    *
FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_mdq_pog; 
""")